# Recurrent Neural Network (RNN)


An **RNN** is a type of neural network made for handling **sequences** — like sentences, time-series data, or audio.  
Unlike normal networks that treat every input independently, RNNs **remember past information** using something called a **hidden state**.

---

###  How It Works
At each time step:

hₜ = activation(Wₓₕ · xₜ + Wₕₕ · hₜ₋₁ + bₕ)
yₜ = Wₕᵧ · hₜ + bᵧ

Where:
- Input → `xₜ`  
- Previous memory → `hₜ₋₁`  
- Updated memory → `hₜ`  
- Output → `yₜ`
`Wₓₕ`, `Wₕₕ`, `Wₕᵧ` → weight matrices  
`bₕ`, `bᵧ` → bias terms  
`activation()` → non-linear function such as *tanh* or *ReLU*


Basically, it reads one input at a time, updates its memory, and produces an output — kind of like how we process words in a sentence.

---

###  Quick PyTorch Example

```python
import torch
import torch.nn as nn

rnn = nn.RNN(input_size=10, hidden_size=20, batch_first=True)
x = torch.randn(5, 3, 10)  # (batch, seq_len, input_size)

output, hidden = rnn(x)
print(output.shape, hidden.shape)


In [1]:
import pandas as pd

df = pd.read_csv('100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


### Tokenization Function

The following function takes a text input, converts it to lowercase, removes certain punctuation marks (`?` and `'`), and then splits the text into individual tokens (words).


In [2]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [3]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

### lets create dictionary 

In [4]:
#vocab
vocab={'<UNK>':0}

In [5]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [6]:
df.apply(build_vocab,axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [7]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [8]:
len(vocab)

324

In [9]:
# vectorization

In [10]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [11]:
text_to_indices("What what is campusx", vocab)

[1, 1, 2, 0]

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader

In [13]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [14]:
dataset = QADataset(df, vocab)

In [15]:
for i in range(10):
    print(dataset[i])

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))
(tensor([1, 2, 3, 4, 5, 8]), tensor([9]))
(tensor([10, 11, 12, 13, 14, 15]), tensor([16]))
(tensor([ 1,  2,  3, 17, 18, 19, 20, 21, 22]), tensor([23]))
(tensor([ 1,  2,  3, 24, 25,  5, 26, 19, 27]), tensor([28]))
(tensor([10, 29,  3, 30, 31]), tensor([32]))
(tensor([ 1,  2,  3, 33, 34,  5, 35]), tensor([36]))
(tensor([ 1,  2,  3, 37, 38, 39, 40]), tensor([41]))
(tensor([42, 43, 44, 45, 46, 47, 48]), tensor([49]))
(tensor([ 1,  2,  3, 50, 51, 19,  3, 45]), tensor([52]))


In [16]:
# creating dataloder

In [17]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [18]:
for question, answer in dataloader:
  print(question, answer)

tensor([[ 78,  79, 288,  81,  19,  14, 289]]) tensor([[85]])
tensor([[1, 2, 3, 4, 5, 6]]) tensor([[7]])
tensor([[ 42, 137,   2, 138,  39, 139]]) tensor([[53]])
tensor([[  1,   2,   3,  17, 115,  83,  84]]) tensor([[116]])
tensor([[ 42, 250, 251, 118, 252, 253]]) tensor([[254]])
tensor([[ 42, 137,   2, 138,  39, 175, 269]]) tensor([[99]])
tensor([[  1,   2,   3, 163, 164, 165,  83,  84]]) tensor([[166]])
tensor([[ 78,  79, 150, 151,  14, 152, 153]]) tensor([[154]])
tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([[128]])
tensor([[  1,   2,   3,  69,   5, 155]]) tensor([[156]])
tensor([[ 42,  18,   2,   3, 281,  12,   3, 282]]) tensor([[205]])
tensor([[ 10,  96,   3, 104, 239]]) tensor([[240]])
tensor([[ 42, 137,   2,  62,  39,   3, 322, 323]]) tensor([[6]])
tensor([[  1,   2,   3,   4,   5, 279]]) tensor([[280]])
tensor([[ 1,  2,  3, 24, 25,  5, 26, 19, 27]]) tensor([[28]])
tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([[49]])
tensor([[  1,   2,   3, 103,   5, 104,  19, 105]])

In [19]:
import torch.nn as nn

##  Understanding nn.Embedding

**nn.Embedding** is like a **lookup table** for discrete items (words, categories, etc.).  
Instead of huge one-hot vectors, each item gets a **dense vector** of numbers which the model can learn during training.



### Quick Theory
- **num_embeddings** -> number of unique items (vocab size).  
- **embedding_dim** -> length of vector for each item (how much info you store per item).  
- Embedding vectors are **trainable** and updated via backprop.  
- Bigger `embedding_dim` -> more capacity to capture patterns, but does **not** directly set final output size.


In [27]:
emb=nn.Embedding(300, embedding_dim=2)
print("output shape :",emb)
print("length of the vector for each item: ",20)
print(question)
print("Max index:", question.max())
print("Min index:", question.min())
print(question.shape)
embed=emb(question)
embed

output shape : Embedding(300, 2)
length of the vector for each item:  20
tensor([[ 42, 137,   2, 226,  12,   3, 227, 228]])
Max index: tensor(228)
Min index: tensor(2)
torch.Size([1, 8])


tensor([[[ 0.0214, -0.1073],
         [-1.3040, -0.4241],
         [ 0.5173, -1.2312],
         [ 1.6234,  1.2932],
         [ 1.0786,  1.3691],
         [-0.3321, -0.7792],
         [-0.9732,  1.0950],
         [-0.1937, -0.0326]]], grad_fn=<EmbeddingBackward0>)

In [28]:
x=nn.Embedding(324, embedding_dim=50)
print(dataset[0][0],dataset[0][0].shape)
a=x(dataset[0][0])
a.shape,a
#x has total 6 elements so 6,50 is shape

tensor([1, 2, 3, 4, 5, 6]) torch.Size([6])


(torch.Size([6, 50]),
 tensor([[-3.4757e-01, -1.1185e+00, -1.8477e+00, -2.2723e-01,  7.1608e-01,
          -1.4467e+00, -7.5168e-01,  2.0945e+00, -1.3286e+00, -3.5295e-01,
          -1.8365e-01, -3.0587e-01, -9.0066e-01, -1.7778e-01, -9.8296e-01,
           2.2085e-03,  6.4598e-01,  3.3977e-01, -1.0724e+00, -7.3836e-01,
          -2.9359e-01, -1.0467e-01, -1.1270e+00, -3.5008e-01,  2.5910e+00,
          -5.0761e-01, -1.1595e+00, -2.2467e+00, -1.3160e+00,  5.9151e-01,
           4.7780e-01,  1.1370e-01, -1.1499e+00, -7.4423e-01, -4.8371e-01,
           1.3575e+00, -2.4155e-01, -2.0856e+00,  5.2163e-01,  1.1638e+00,
           1.5997e-01, -1.0355e+00, -1.9128e-01, -4.1502e-01, -3.1690e-01,
          -2.3535e+00,  1.3392e+00,  5.9038e-01, -1.5578e+00,  9.9716e-01],
         [ 1.1663e+00, -6.5174e-01,  1.2807e-01, -4.9102e-01, -5.9433e-01,
           3.2379e-01,  6.2405e-01, -9.3339e-01, -2.1418e-01, -4.0515e-01,
          -1.4433e+00,  6.2684e-01,  1.2646e+00,  5.5050e-01,  1.3649e-01,
  

In [29]:
y=nn.RNN(50,64)
b=y(a)
b

(tensor([[ 0.0520, -0.1965, -0.0261,  0.6318, -0.7570,  0.5411, -0.2152,  0.7473,
          -0.6137, -0.0844,  0.1601, -0.3194,  0.2334,  0.4295, -0.3672, -0.1864,
           0.8404,  0.1848, -0.3418,  0.7027,  0.4814, -0.9167,  0.6647,  0.6688,
          -0.5409, -0.0819,  0.2490,  0.3889, -0.0545,  0.4940,  0.8813, -0.3146,
           0.4582, -0.0628,  0.8724,  0.0480, -0.0362, -0.4824, -0.2905,  0.1230,
          -0.2359,  0.1772,  0.3179, -0.5451,  0.3632, -0.6342, -0.4603, -0.1714,
           0.4366,  0.0282,  0.2910,  0.0807,  0.3925, -0.6108,  0.1224,  0.5332,
           0.2137, -0.2955,  0.4274,  0.2884, -0.0740, -0.0875, -0.0292,  0.5153],
         [ 0.5931, -0.3844,  0.2177, -0.0566,  0.2949, -0.0990, -0.2959,  0.7536,
           0.8415,  0.7641,  0.4095,  0.3720, -0.6354,  0.1634, -0.2217, -0.1526,
           0.3148,  0.5211,  0.5918, -0.1017,  0.2897,  0.5560, -0.0474, -0.4605,
          -0.2000,  0.4908,  0.7854,  0.2016, -0.3909, -0.7083, -0.6721,  0.5488,
          -0.01

In [30]:
b[0].shape,b[1].shape

(torch.Size([6, 64]), torch.Size([1, 64]))

In [31]:
#b[0] is hidden state
#b[1] is output state

In [32]:
z = nn.Linear(64, 324)
z(b[1])

tensor([[ 0.2256, -0.1486, -0.4389, -0.0932, -0.1999,  0.7010, -0.5429,  0.2647,
         -0.5593,  0.1903,  0.5463, -0.1624, -0.3073,  0.1148,  0.3493, -0.0817,
         -0.4266, -0.1426, -0.2778, -0.3442, -0.0096,  0.5652, -0.3062, -0.4221,
         -0.0838,  0.2304,  0.4136, -0.0798,  0.0562,  0.2885, -0.0446, -0.1609,
         -0.5340,  0.0988,  0.0950, -0.3094, -0.3229,  0.0302, -0.1258,  0.3998,
         -0.9187,  0.0849, -0.2420, -0.0277, -0.1401, -0.1322,  0.0039, -0.2287,
          0.3470, -0.2731,  0.4262,  0.1097,  0.1413, -0.1710,  0.5327,  0.8139,
         -0.2637, -0.5440, -0.1391,  0.3175, -0.2680, -0.0501,  0.1702,  0.0079,
         -0.4135, -0.0703,  0.2054, -0.0467,  0.1328,  0.1309, -0.1966,  0.0540,
         -0.2312, -0.2224, -0.5446,  0.3459, -0.0352, -0.3395, -0.1687, -0.0607,
         -0.3148,  0.3370, -0.2866,  0.1546,  0.7396,  0.4275, -0.3232, -0.0166,
         -0.2199,  0.4315, -0.1319,  0.1306, -0.2766, -0.0469,  0.0695,  0.1822,
         -0.5403, -0.3861,  

In [33]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [34]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [35]:
learning_rate = 0.001
epochs = 20

In [36]:
#create model
model = SimpleRNN(len(vocab))

In [37]:
#set loss() and optimizer() 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [38]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 521.218998
Epoch: 2, Loss: 446.045166
Epoch: 3, Loss: 365.759383
Epoch: 4, Loss: 307.659921
Epoch: 5, Loss: 258.092196
Epoch: 6, Loss: 210.375345
Epoch: 7, Loss: 167.936773
Epoch: 8, Loss: 131.887758
Epoch: 9, Loss: 100.793404
Epoch: 10, Loss: 77.591450
Epoch: 11, Loss: 59.582944
Epoch: 12, Loss: 47.179898
Epoch: 13, Loss: 37.901377
Epoch: 14, Loss: 31.111751
Epoch: 15, Loss: 25.524438
Epoch: 16, Loss: 21.558590
Epoch: 17, Loss: 18.303061
Epoch: 18, Loss: 15.787401
Epoch: 19, Loss: 13.426176
Epoch: 20, Loss: 11.856471


In [39]:
def predict(model, question, threshold=0.02):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [40]:
list(vocab.keys())[7]

'paris'

In [41]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [42]:
predict(model, "What planet our solar system?")

jupiter
